 
# Alpha and Beta Diversity

Different higher-level measures are often used to describe the microbiome in a sample. These do not provide information on changes in the abundance of specific taxa but allow us to access a broader change or difference in the composition of microorganisms. Alpha and beta diversity are examples of such measures.

Different measures exist to estimate diversity within a single sample, jointly called alpha diversity. The different measures reflect the richness (number) or distribution (evenness) of a microbial sample or aim to reflect a combination of both properties.

Rarefaction curves are often used when calculating alpha diversity indices because increasing numbers of sequenced taxa allow increasingly accurate estimates of total population diversity. Rarefaction curves can therefore be used to estimate the full sample richness, as compared to the observed sample richness.

While alpha diversity is a measure of microbiome diversity applicable to a single sample, beta diversity is a measure of the similarity or dissimilarity of two communities. As for alpha diversity, many indices exist, each reflecting different aspects of community heterogeneity. Key differences relate to how the indices value variation in rare species if they consider presence/absence only or incorporate abundance, and how they interpret shared absence. Bray-Curtis dissimilarity is a popular measure that considers both size (overall abundance per sample) and shape (abundance of each taxon) of the communities (Bray, 1957). Beta diversity is an essential measure for many popular statistical methods in ecology, such as ordination-based methods, and is widely used for studying the association between environmental variables and microbial composition.

In summary, alpha diversity measures can be seen as a summary statistic of a single population (within-sample diversity), while beta diversity measures are estimates of similarity or dissimilarity between populations (between samples).

**Source**: (https://biomcare.com/info/key-terms-in-microbiome-projects/)

### STEP : Diversity Analysis

Using QIIME2 to create diversity analisys graphs and calculations.

- [QIIME2 Workflow Overview](https://docs.qiime2.org/2022.8/tutorials/overview/)


#### Methods
- [diversity](https://docs.qiime2.org/2022.8/plugins/available/diversity/)
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/)
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/)
- [diversity beta](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta/)
- [diversity core_metrics](https://docs.qiime2.org/2022.8/plugins/available/diversity/core-metrics/)
- [diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/)
- [diversity beta_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-group-significance/)
- [feature_table core_features](https://docs.qiime2.org/2022.8/plugins/available/feature-table/core-features/)
- [feature_table summarize](https://docs.qiime2.org/2022.8/plugins/available/feature-table/summarize/)
- [taxa filter-table](https://docs.qiime2.org/2022.8/plugins/available/taxa/filter-table/)
- [taxa collapse](https://docs.qiime2.org/2022.8/plugins/available/taxa/collapse/)

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.phylogeny.pipelines import align_to_tree_mafft_fasttree

from qiime2.plugins.diversity.pipelines import alpha
from qiime2.plugins.diversity.pipelines import beta
from qiime2.plugins.diversity.pipelines import core_metrics
from qiime2.plugins.diversity.pipelines import alpha_phylogenetic

from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.diversity.visualizers import alpha_correlation
from qiime2.plugins.diversity.visualizers import beta_rarefaction

from qiime2.plugins.taxa.methods import filter_table
from qiime2.plugins.taxa.methods import collapse

from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.visualizers import core_features

from qiime2.plugins.alignment.methods import mafft


import matplotlib.pyplot as plt

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
base_dir = os.path.join('/', 'home')
metadata_file = os.path.abspath(os.path.join(base_dir, 'data', 'metadatada.tsv'))
experiment_name = ''
class_col = ''
replace_files = False

In [3]:
# Parameters
experiment_name = "thayane-PM"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-thayane"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/manifest.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/metadata.tsv"
class_col = "menopausa-age-qcut"
classifier_file = (
    "/home/lauro/nupeb/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
)
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6


In [27]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [6]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')
dada2_reps_path = os.path.join(qiime_folder, 'dada2-reps.qza')
dada2_stat_path = os.path.join(qiime_folder, 'dada2-stat.qza')

# Input - Taxonaomic Artifacts
taxonomy_path = os.path.join(qiime_folder, 'metatax.qza')

# Create folder to store Alpha files
alpha_path = os.path.join(qiime_folder, 'alpha-analysis')
if not os.path.exists(alpha_path):
    os.makedirs(alpha_path)
    print(f'The new directory is created in {alpha_path}')
    
# Create folder to store Beta files
beta_path = os.path.join(qiime_folder, 'beta-analysis')
if not os.path.exists(beta_path):
    os.makedirs(beta_path)
    print(f'The new directory is created in {beta_path}')

# Output -Diversity Artifacts
alpha_diversity_path = os.path.join(alpha_path, 'alpha-diversity.qza')
alpha_diversity_view_path = os.path.join(alpha_path, 'alpha-diversity.qzv')
beta_diversity_path = os.path.join(beta_path, 'beta-diversity.qza')
beta_diversity_view_path = os.path.join(beta_path, 'beta-diversity.qzv')

In [28]:
def filter_and_collapse(tab, seqs, tax, meta, lvl, exclude=True, exclude_list='uncultured,unidentified,metagenome'):
    from qiime2.plugins.taxa.methods import collapse
    from qiime2.plugins.taxa.methods import filter_table
    from qiime2.plugins.feature_table.methods import filter_seqs
    from qiime2.plugins.feature_table.visualizers import summarize
    
    to_include = ('d', 'p', 'c', 'o', 'f', 'g', 's')[lvl-1]
    to_include += '__'
    to_exclude = exclude_list if exclude else None
    
    filtered_tabs = filter_table(
        table=tab, 
        taxonomy=tax,
        include=to_include,
        exclude=to_exclude,
        mode='contains').filtered_table
    
    filtered_seqs = filter_seqs(
        data = seqs,
        table = filtered_tabs,
    ).filtered_data
    
    collapsed_table = collapse(table=filtered_tabs, taxonomy=tax, level=lvl).collapsed_table
    collapsed_table_view = summarize(table=collapsed_table, sample_metadata=meta).visualization
    
    return collapsed_table, collapsed_table_view, filtered_seqs

## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [30]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)
tabs_df = tabs.view(Metadata).to_dataframe().T

# FeatureData[Sequence]
reps = Artifact.load(dada2_reps_path)

# FeatureData[Taxonomy]
tax = Artifact.load(taxonomy_path)

In [31]:
categorical_columns = metadata_qa.filter_columns(column_type='categorical')
numeric_columns = metadata_qa.filter_columns(column_type='numeric')

In [8]:
# lvl = 7
# exclude = True
# tabs, collapsed_table_view, reps = filter_and_collapse(
#                     tabs, reps, tax, metadata_qa, 
#                     lvl=lvl,
#                     exclude=exclude, 
#                     exclude_list='uncultured,unidentified,metagenome')
# collapsed_table_view

## Alpha diversity analysis

#### Reference
- [The Use and Types of Alpha-Diversity Metrics in Microbial NGS](https://www.cd-genomics.com/microbioseq/the-use-and-types-of-alpha-diversity-metrics-in-microbial-ngs.html)
- [Alpha diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.alpha.html)

#### Methods
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/): Computes a user-specified alpha diversity metric for all samples in a
feature table.
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/): Computes a user-specified phylogenetic alpha diversity metric for all
samples in a feature table.
- [diversity alpha_correlation](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-correlation/): Determine whether numeric sample metadata columns are correlated with alpha diversity.
- [diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/): Visually and statistically compare groups of alpha diversity values.

### Compute Alpha Diversity vectors
- [diversity alpha](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha/): Computes a user-specified alpha diversity metric for all samples in a feature table.
- [Alpha diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.alpha.html)
 - Choices: ('ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_features', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong')

In [9]:
metrics = ('ace', 'berger_parker_d', 'brillouin_d', 'chao1', 'chao1_ci', 'dominance', 'doubles', 'enspie', 'esty_ci', 'fisher_alpha', 'gini_index', 'goods_coverage', 'heip_e', 'kempton_taylor_q', 'lladser_pe', 'margalef', 'mcintosh_d', 'mcintosh_e', 'menhinick', 'michaelis_menten_fit', 'observed_features', 'osd', 'pielou_e', 'robbins', 'shannon', 'simpson', 'simpson_e', 'singles', 'strong')
alpha_diversities = dict()
for metric in metrics:
    print(f"Calculating alpha diversity: {metric}")
    try:
        alpha_diversity = alpha(table=tabs, metric=metric).alpha_diversity
        alpha_diversities[metric] = alpha_diversity
        # Save SampleData[AlphaDiversity] Artifact
        file_path = os.path.join(alpha_path, f'alpha-values-{metric}.qza')
        alpha_diversity.save(file_path)
        print(f"DONE: Calculating alpha diversity: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha diversity: {metric}")
        print(e)

Calculating alpha diversity: ace
ERROR: Calculating alpha diversity: ace
index 1 is out of bounds for axis 0 with size 1
Calculating alpha diversity: berger_parker_d
DONE: Calculating alpha diversity: berger_parker_d
Calculating alpha diversity: brillouin_d
DONE: Calculating alpha diversity: brillouin_d
Calculating alpha diversity: chao1


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:55: RuntimeWarning: invalid value encountered in long_scalars
  return counts.max() / counts.sum()
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:95: RuntimeWarning: invalid value encountered in double_scalars
  return (gammaln(n + 1) - gammaln(nz + 1).sum()) / n


DONE: Calculating alpha diversity: chao1
Calculating alpha diversity: chao1_ci
DONE: Calculating alpha diversity: chao1_ci
Calculating alpha diversity: dominance


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_chao1.py:217: RuntimeWarning: invalid value encountered in long_scalars
  P = np.exp(-n / s)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:138: RuntimeWarning: invalid value encountered in true_divide
  freqs = counts / counts.sum()


DONE: Calculating alpha diversity: dominance
Calculating alpha diversity: doubles
DONE: Calculating alpha diversity: doubles
Calculating alpha diversity: enspie
DONE: Calculating alpha diversity: enspie
Calculating alpha diversity: esty_ci


...

/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:480: RuntimeWarning: divide by zero encountered in log
  return (observed_otus(counts) - 1) / np.log(counts.sum())
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:532: RuntimeWarning: invalid value encountered in double_scalars
  return (n - u) / (n - np.sqrt(n))
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:579: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:618: RuntimeWarning: invalid value encountered in true_divide
  return observed_otus(counts) / np.sqrt(counts.sum())


DONE: Calculating alpha diversity: menhinick
Calculating alpha diversity: michaelis_menten_fit
DONE: Calculating alpha diversity: michaelis_menten_fit
Calculating alpha diversity: observed_features
DONE: Calculating alpha diversity: observed_features
Calculating alpha diversity: osd
DONE: Calculating alpha diversity: osd
Calculating alpha diversity: pielou_e
DONE: Calculating alpha diversity: pielou_e
Calculating alpha diversity: robbins


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:793: RuntimeWarning: divide by zero encountered in log
  return shannon(counts, base=np.e) / np.log(observed_otus(counts))
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:829: RuntimeWarning: invalid value encountered in long_scalars
  return singles(counts) / counts.sum()


DONE: Calculating alpha diversity: robbins
Calculating alpha diversity: shannon
DONE: Calculating alpha diversity: shannon
Calculating alpha diversity: simpson
DONE: Calculating alpha diversity: simpson
Calculating alpha diversity: simpson_e
DONE: Calculating alpha diversity: simpson_e
Calculating alpha diversity: singles
DONE: Calculating alpha diversity: singles
Calculating alpha diversity: strong
DONE: Calculating alpha diversity: strong


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:1021: RuntimeWarning: invalid value encountered in true_divide
  return (sorted_sum / n - (i / s)).max()
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/diversity/alpha/_base.py:1021: RuntimeWarning: divide by zero encountered in true_divide
  return (sorted_sum / n - (i / s)).max()


### Create Phylogenetic inference

- [alignment align_to_tree_mafft_fasttree](https://docs.qiime2.org/2022.8/plugins/available/phylogeny/align-to-tree-mafft-fasttree/): Build a phylogenetic tree using fasttree and mafft alignment

This pipeline will start by creating a sequence alignment using MAFFT,
after which any alignment columns that are phylogenetically uninformative
or ambiguously aligned will be removed (masked). The resulting masked
alignment will be used to infer a phylogenetic tree and then subsequently
rooted at its midpoint. Output files from each step of the pipeline will be
saved. This includes both the unmasked and masked MAFFT alignment from
q2-alignment methods, and both the rooted and unrooted phylogenies from
q2-phylogeny methods.


Returns
- alignment : FeatureData[AlignedSequence] : The aligned sequences.
- masked_alignment : FeatureData[AlignedSequence] : The masked alignment.
- tree : Phylogeny[Unrooted] : The unrooted phylogenetic tree.
- rooted_tree : Phylogeny[Rooted] : The rooted phylogenetic tree.

In [10]:
mafft_alignment, mafft_masked_alignment, mafft_tree, mafft_rooted_tree = align_to_tree_mafft_fasttree(
    sequences=reps, n_threads=6, )

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 6 /tmp/qiime2-archive-1sdzx6ya/13d02254-ae40-454f-83ff-29cf80d39c56/data/dna-sequences.fasta

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTreeMP -quote -nt /tmp/qiime2-archive-kc23nh6e/ffe36140-d4ea-4956-b157-b6f9964adb66/data/aligned-dna-sequences.fasta



### Compute Alpha Diversity (Phylogeny)
- [diversity alpha_phylogenetic](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-phylogenetic/): Computes a user-specified phylogenetic alpha diversity metric for all samples in a feature table.
- Metrics: Choices ('faith_pd')

In [11]:
metrics = ('faith_pd', )
alpha_diversities_phylogenetic = dict()
for metric in metrics:
    print(f"Calculating alpha diversity: {metric}")
    try:
        alpha_diversity = alpha_phylogenetic(table=tabs, phylogeny=mafft_rooted_tree, metric=metric).alpha_diversity
        alpha_diversities_phylogenetic[metric] = alpha_diversity
        # Save Artifact
        file_path = os.path.join(alpha_path, f'alpha-phylogeny-{metric}.qza')
        alpha_diversity.save(file_path)
        print(f"DONE: Calculating alpha phylogeny: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha phylogeny: {metric}")

Calculating alpha diversity: faith_pd
DONE: Calculating alpha phylogeny: faith_pd


### Alpha diversity correlation

This method only process `numeric` columns.


In [12]:
methods = ('spearman', 'pearson')
numerics_cols = metadata_qa.filter_columns(column_type='numeric')
if numerics_cols.column_count > 0:
    for metric, alpha_values in alpha_diversities.items():
        for method in methods:
            try:
                corr_view = alpha_correlation(alpha_diversity=alpha_values, metadata=numerics_cols, 
                                          method=method, intersect_ids=True).visualization
                view_path = os.path.join(alpha_path, f'alpha-correlation-{metric}-{method}.qzv')
                corr_view.save(view_path)
                corr_view
                print(f"DONE: Calculating alpha correlation: {metric} {method}")
            except Exception as e:
                print(f"ERROR: Calculating alpha correlation: {metric} {method}")

DONE: Calculating alpha correlation: berger_parker_d spearman
ERROR: Calculating alpha correlation: berger_parker_d pearson
DONE: Calculating alpha correlation: brillouin_d spearman
ERROR: Calculating alpha correlation: brillouin_d pearson
DONE: Calculating alpha correlation: chao1 spearman
DONE: Calculating alpha correlation: chao1 pearson
ERROR: Calculating alpha correlation: chao1_ci spearman
ERROR: Calculating alpha correlation: chao1_ci pearson
DONE: Calculating alpha correlation: dominance spearman
ERROR: Calculating alpha correlation: dominance pearson
DONE: Calculating alpha correlation: doubles spearman
DONE: Calculating alpha correlation: doubles pearson
DONE: Calculating alpha correlation: enspie spearman
ERROR: Calculating alpha correlation: enspie pearson
ERROR: Calculating alpha correlation: esty_ci spearman
ERROR: Calculating alpha correlation: esty_ci pearson
DONE: Calculating alpha correlation: fisher_alpha spearman
DONE: Calculating alpha correlation: fisher_alpha pea

## Alpha diversity comparisons

Visually and statistically compare groups of alpha diversity values.

[diversity alpha_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/alpha-group-significance/)

In [13]:
for metric, alpha_values in alpha_diversities.items():
    print(f"Processing alpha_group_significance: {metric}")
    try:
        significance_view = alpha_group_significance(alpha_diversity=alpha_values, metadata=metadata_qa).visualization
        view_path = os.path.join(alpha_path, f'alpha-group-significance-{metric}.qzv')
        significance_view.save(view_path)
        significance_view
        print(f"DONE: Calculating alpha group significance: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating alpha group significance: {metric}")

Processing alpha_group_significance: berger_parker_d
DONE: Calculating alpha group significance: berger_parker_d
Processing alpha_group_significance: brillouin_d
DONE: Calculating alpha group significance: brillouin_d
Processing alpha_group_significance: chao1
DONE: Calculating alpha group significance: chao1
Processing alpha_group_significance: chao1_ci
ERROR: Calculating alpha group significance: chao1_ci
Processing alpha_group_significance: dominance
DONE: Calculating alpha group significance: dominance
Processing alpha_group_significance: doubles
DONE: Calculating alpha group significance: doubles
Processing alpha_group_significance: enspie
DONE: Calculating alpha group significance: enspie
Processing alpha_group_significance: esty_ci
ERROR: Calculating alpha group significance: esty_ci
Processing alpha_group_significance: fisher_alpha
DONE: Calculating alpha group significance: fisher_alpha
Processing alpha_group_significance: gini_index
DONE: Calculating alpha group significance:

## Beta diversity analysis

#### Reference
- [diversity beta](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta/): Computes a user-specified beta diversity metric for all pairs of samples in a feature table.
- [Beta diversity metrics](http://scikit-bio.org/docs/0.2.0/generated/skbio.diversity.beta.html)

- Metric Choices('aitchison', 'braycurtis', 'canberra', 'canberra_adkins', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule')

In [14]:
metrics = ('aitchison', 'braycurtis', 'canberra', 'canberra_adkins', 'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean', 'hamming', 'jaccard', 'jensenshannon', 'kulsinski', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule')
beta_diversities = dict()
for metric in metrics:
    print(f"Calculating beta diversity: {metric}")
    try:
        beta_diversity = beta(table=tabs, metric=metric, n_jobs=6, pseudocount=1).distance_matrix
        beta_diversities[metric] = beta_diversity
        # Save SampleData[BetaDiversity] Artifact
        file_path = os.path.join(beta_path, f'beta-values-{metric}.qza')
        beta_diversity.save(file_path)
        print(f"DONE: Calculating beta diversity: {metric}")
    except Exception as e:
        print(f"ERROR: Calculating beta diversity: {metric}")

Calculating beta diversity: aitchison
DONE: Calculating beta diversity: aitchison
Calculating beta diversity: braycurtis
ERROR: Calculating beta diversity: braycurtis
Calculating beta diversity: canberra
DONE: Calculating beta diversity: canberra
Calculating beta diversity: canberra_adkins
ERROR: Calculating beta diversity: canberra_adkins
Calculating beta diversity: chebyshev


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/q2_diversity_lib/beta.py:67: RuntimeWarning: divide by zero encountered in true_divide
  return (1. / nnz) * np.sum(np.abs(x_ - y_) / (x_ + y_))
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/q2_diversity_lib/beta.py:67: RuntimeWarning: invalid value encountered in double_scalars
  return (1. / nnz) * np.sum(np.abs(x_ - y_) / (x_ + y_))


DONE: Calculating beta diversity: chebyshev
Calculating beta diversity: cityblock
DONE: Calculating beta diversity: cityblock
Calculating beta diversity: correlation
ERROR: Calculating beta diversity: correlation
Calculating beta diversity: cosine
ERROR: Calculating beta diversity: cosine
Calculating beta diversity: dice


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric dice
  warnings.warn(msg, DataConversionWarning)


ERROR: Calculating beta diversity: dice
Calculating beta diversity: euclidean
DONE: Calculating beta diversity: euclidean
Calculating beta diversity: hamming
DONE: Calculating beta diversity: hamming
Calculating beta diversity: jaccard


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/scipy/spatial/distance.py:1291: RuntimeWarning: invalid value encountered in true_divide
  q = q / np.sum(q, axis=axis, keepdims=True)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/scipy/spatial/distance.py:1290: RuntimeWarning: invalid value encountered in true_divide
  p = p / np.sum(p, axis=axis, keepdims=True)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric kulsinski
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to

DONE: Calculating beta diversity: jaccard
Calculating beta diversity: jensenshannon
ERROR: Calculating beta diversity: jensenshannon
Calculating beta diversity: kulsinski
ERROR: Calculating beta diversity: kulsinski
Calculating beta diversity: matching
DONE: Calculating beta diversity: matching
Calculating beta diversity: minkowski
DONE: Calculating beta diversity: minkowski
Calculating beta diversity: rogerstanimoto
DONE: Calculating beta diversity: rogerstanimoto
Calculating beta diversity: russellrao
ERROR: Calculating beta diversity: russellrao
Calculating beta diversity: seuclidean
DONE: Calculating beta diversity: seuclidean
Calculating beta diversity: sokalmichener


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric rogerstanimoto
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric russellrao
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric sokalmichener
  warnings.warn(msg, DataConversionWarning)


DONE: Calculating beta diversity: sokalmichener
Calculating beta diversity: sokalsneath
ERROR: Calculating beta diversity: sokalsneath
Calculating beta diversity: sqeuclidean
DONE: Calculating beta diversity: sqeuclidean
Calculating beta diversity: yule
DONE: Calculating beta diversity: yule


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric sokalsneath
  warnings.warn(msg, DataConversionWarning)
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1776: DataConversionWarning: Data was converted to boolean for metric yule
  warnings.warn(msg, DataConversionWarning)


### Beta group significance

- [diversity beta_group_significance](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-group-significance/): Determine whether groups of samples are significantly different from one another using a permutation-based statistical test.
- Marti J Anderson. A new method for non-parametric multivariate analysis of variance. Austral ecology, 26(1):32–46, 2001. doi:https://doi.org/10.1111/j.1442-9993.2001.01070.pp.x.

In [15]:
methods = ('permanova', 'anosim', 'permdisp')
for method in methods:
    for metric, beta_diversity in beta_diversities.items():
        print(f'Calculating beta group significance with method {method} and metric {metric}')
        try:
            beta_view = beta_group_significance(distance_matrix=beta_diversity, 
                                                metadata=metadata_qa.get_column(class_col), 
                                                pairwise=True, method=method).visualization
            view_name = os.path.join(beta_path, f'beta-group-significance-{metric}-{method}.qzv')
            beta_view.save(view_name)
            print(f"DONE: Calculating beta group significance: {method} {metric}")
        except Exception as e:
            print(f"ERROR: Calculating beta group significance: {method} {metric}")

Calculating beta group significance with method permanova and metric aitchison
DONE: Calculating beta group significance: permanova aitchison
Calculating beta group significance with method permanova and metric canberra
DONE: Calculating beta group significance: permanova canberra
Calculating beta group significance with method permanova and metric chebyshev
DONE: Calculating beta group significance: permanova chebyshev
Calculating beta group significance with method permanova and metric cityblock
DONE: Calculating beta group significance: permanova cityblock
Calculating beta group significance with method permanova and metric euclidean
DONE: Calculating beta group significance: permanova euclidean
Calculating beta group significance with method permanova and metric hamming
DONE: Calculating beta group significance: permanova hamming
Calculating beta group significance with method permanova and metric jaccard
DONE: Calculating beta group significance: permanova jaccard
Calculating beta

/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -17003.01500386638 and the largest is 27659.412214200038.
  warn(
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for mor

DONE: Calculating beta group significance: permdisp canberra
Calculating beta group significance with method permdisp and metric chebyshev


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -57123817.81418646 and the largest is 1082773330.6122975.
  warn(
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for mor

DONE: Calculating beta group significance: permdisp chebyshev
Calculating beta group significance with method permdisp and metric cityblock


...

/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -1.6621932617873482e+17 and the largest is 1.400981154851075e+18.
  warn(
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section

DONE: Calculating beta group significance: permdisp sqeuclidean
Calculating beta group significance with method permdisp and metric yule


/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -5.357252904034342 and the largest is 18.45907949108593.
  warn(
/home/lauro/anaconda3/envs/qiime2-2021.11/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more

DONE: Calculating beta group significance: permdisp yule


<Figure size 432x288 with 0 Axes>

### Beta group Rarefaction

- [diversity beta_rarefaction](https://docs.qiime2.org/2022.8/plugins/available/diversity/beta-rarefaction/): Repeatedly rarefy a feature table to compare beta diversity results within a given rarefaction depth.  For a given beta diversity metric, this visualizer will provide: an Emperor jackknifed PCoA plot, samples clustered by UPGMA or neighbor joining with support calculation, and a heatmap showing the correlation between rarefaction trials of that beta diversity metric.